## ВМА. Лабораторная 3. Отчет
### Гамезо Валерия

### Постановка задачи

Разработать программу численного решения методом прогонки СЛАУ вида $Ay=f$ (вида (1) в файле «Метод прогонки»).
Матрицу системы задать следующим образом: 

$$
A = \left( \begin{array}{}
m & m - 1 & & & & \\
-k & m + k & m & & & \\
& -k & m + k + 1 & m + 1 & & \\
& & \dotsm & \dotsm & \dotsm & \\
& & & -k & m + k + N - 2 & m + N - 2 \\
& & & & -k & m + k + N - 1
\end{array} \right) .
$$

Правую часть $f$ задать умножением матрицы $A$ на вектор $y=(1, 2, ... , N+1):  f=Ay$.
Для вычислений выбрать параметры: 

- $m$ – номер в списке студенческой группы; 
- $N+1$  – (порядок матрицы) одно из чисел в пределах от $10$ до $12$;
- $k$ – номер студенческой группы. 

Программно реализовать (в качестве языка программирования выбрать C или C++) вычисления для рассматриваемого примера. Для вычислений использовать тип float. 
В отчете должны быть представлены:

1. Преобразованная матрица после прямой прогонки 
2. Вектор приближённого решения $y^*$.
3. Относительная погрешность вида $\large \frac{\|y - y^*\|_{\textrm{I}}}{\|y\|_{\textrm{I}}}$ , где $y$ – точное решение.

Содержание работы должно включать следующие пункты. 
1. Постановка задачи.
2. Входные данные.
3. Листинг программы. Обязательны подробные комментарии.
4. Выходные данные. 
5. Выводы.


### Входные данные

$N + 1 = 12$

$m = 5$

$k = 1$

$$
A = \left( 
\begin{array}{rrrrrrrrrrrr|r}
    5 &    4 & & & & & & & & & & &   13 \\
   -1 &    6 &    5 & & & & & & & & & &   26 \\
 &   -1 &    7 &    6 & & & & & & & & &   43 \\
 & &   -1 &    8 &    7 & & & & & & & &   64 \\
 & & &   -1 &    9 &    8 & & & & & & &   89 \\
 & & & &   -1 &   10 &    9 & & & & & &  118 \\
 & & & & &   -1 &   11 &   10 & & & & &  151 \\
 & & & & & &   -1 &   12 &   11 & & & &  188 \\
 & & & & & & &   -1 &   13 &   12 & & &  229 \\
 & & & & & & & &   -1 &   14 &   13 & &  274 \\
 & & & & & & & & &   -1 &   15 &   14 &  323 \\
 & & & & & & & & & &   -1 &   16 &  181 \\
\end{array}
\right)
$$

### Листинг программы

Класс, содержащий двумерную матрицу

```c++
struct Matrix {
public:

	const int width_, height_;

	Matrix(int w, int h) : width_(w), height_(h) { // создание матрицы высоты h и шириной w
        	a = new float*[height_];

        for (int i = 0; i < height_; ++i) {
            a[i] = new float[width_];
        }
    }
    
	~Matrix() {
        for (int i = 0; i < height_; ++i) {
            delete[] a[i];
        }
        delete[] a;
    }

	float* operator[] (int i) {
        return a[i];
    }

private:
	float** a;
}; 
```

Класс LES - linear equations system, то есть система линейных уравнений. 
Size - размер матрицы, m, k - заданные условиями переменные

```c++
LES::LES(int size, int m, int k) // создание СЛАУ: трехдиагональной матрицы и столбца свободных членов
	: n_(size), coefs(n_, n_), constTerms(1, n_), k_(k), m_(m) {
	generateCoefs(); // генерация трехдиагональной матрицы
	generateCTerms(); // генерация столбца свободных членов

	state = "LinearEquationsSystem";
}

void LES::forwardSweep() { // прямая прогонка
	for (int i = 0; i < n_ - 1; ++i) {
		constTerms[i][0] /= coefs[i][i];

		coefs[i][i + 1] /= coefs[i][i];
		coefs[i][i] = 1;

		coefs[i + 1][i + 1] -= coefs[i][i + 1] * coefs[i + 1][i];
		constTerms[i + 1][0] -= constTerms[i][0] * coefs[i + 1][i];
	}

	constTerms[n_ - 1][0] /= coefs[n_ - 1][n_ - 1];

	state = "ForwardSweep";
}

void LES::backSubstitution() { // обратная прогонка
	for (int i = n_ - 1; i > 0; --i) {
		constTerms[i - 1][0] -= coefs[i - 1][i] * constTerms[i][0];
	}

	state = "ApproximateSolution";
}

float LES::relativeError() { // вычисление относительной погрешности решения
	float num = 0, den = 0;

	for (int i = 0; i < n_; ++i) {
		num += abs(constTerms[i][0] - (1 + i));
		den += (1 + i);
	}

	return num / den * 100;
}

void LES::generateCoefs() { // генерация коэффициентов трехдиагональной матрицы
	coefs[0][0] = m_;

	for (int i = 0; i < n_ - 1; ++i) {
		coefs[i + 1][i + 1] = m_ + k_ + i;
		coefs[i][i + 1] = m_ + i - 1;
		coefs[i + 1][i] = -1 * k_;
	}
}

void LES::generateCTerms() { // генерация столбца свободных членов
	constTerms[0][0] = coefs[0][0] + 2 * coefs[0][1];

	for (int i = 1; i < n_ - 1; ++i) {
		constTerms[i][0] = i * coefs[i][i - 1] + (i + 1) * coefs[i][i] + (i + 2) * coefs[i][i + 1];
	}

	constTerms[n_ - 1][0] = coefs[n_ - 1][n_ - 2] * (n_ - 1) + coefs[n_ - 1][n_ - 1] * n_;
}
```

### Выходные данные

Полученная двухдиагональная матрица после прямой прогонки:

$$
A = \left(
\begin{array}{rrrrrrrrrrrr|r}
 1 &   0.8000 & & & & & & & & & & &   2.6000 \\
 & 1 &   0.7353 & & & & & & & & & &   4.2059 \\
 & & 1 &   0.7757 & & & & & & & & &   6.1027 \\
 & & & 1 &   0.7977 & & & & & & & &   7.9883 \\
 & & & & 1 &   0.8165 & & & & & & &   9.8991 \\
 & & & & & 1 &   0.8321 & & & & & &  11.8244 \\
 & & & & & & 1 &   0.8452 & & & & &  13.7613 \\
 & & & & & & & 1 &   0.8564 & & & &  15.7072 \\
 & & & & & & & & 1 &   0.8660 & & &  17.6603 \\
 & & & & & & & & & 1 &   0.8745 & &  19.6192 \\
 & & & & & & & & & & 1 &   0.8819 &  21.5830 \\
 & & & & & & & & & & & 1 &  12.0000 \\
\end{array}
\right)
$$

Полученный столбец приближенных решений $Y^*$

$$
Y^* = \left(
\begin{array}{}
   1.000000119 \\
   1.999999762 \\
   3.000000000 \\
   4.000000000 \\
   5.000000000 \\
   6.000000477 \\
   7.000000000 \\
   7.999999046 \\
   9.000000954 \\
   9.999999046 \\
  11.000000000 \\
  12.000000000 \\
\end{array}
\right)
$$

Относительная погрешность:

$$ \large
\frac{\|y - y^*\|_{\textrm{I}}}{\|y\|_{\textrm{I}}} = 0.000691\%
$$

### Вывод

Для трехдиагональной матрицы метод прогонки, в отличии от метода Гаусса, имеет линейную сложность, что сокращает затраты на вычисление решения. Также алгоритм позволяет получать решения СЛАУ с малой относительной погрешностью, возникающие в приложениях, следовательно, даный алгоритм имеет ценность в прикладных вычислениях.